# Imports

In [1]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from difflib import get_close_matches
from difflib import SequenceMatcher
from functools import reduce
import re
import random
from random import randint

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Preparation

In [2]:
!wget -O menu.csv https://drive.google.com/uc?id=17Ol2U4QzXBPppIMzdIXhVtHJB13OQulz&export=download

--2022-05-22 20:49:36--  https://drive.google.com/uc?id=17Ol2U4QzXBPppIMzdIXhVtHJB13OQulz
Resolving drive.google.com (drive.google.com)... 173.194.218.100, 173.194.218.113, 173.194.218.139, ...
Connecting to drive.google.com (drive.google.com)|173.194.218.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gb6bheb0u91pe997sb30ppcf2dvp9iee/1653252525000/10778730237253388309/*/17Ol2U4QzXBPppIMzdIXhVtHJB13OQulz [following]
--2022-05-22 20:49:36--  https://doc-0s-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gb6bheb0u91pe997sb30ppcf2dvp9iee/1653252525000/10778730237253388309/*/17Ol2U4QzXBPppIMzdIXhVtHJB13OQulz
Resolving doc-0s-18-docs.googleusercontent.com (doc-0s-18-docs.googleusercontent.com)... 172.217.193.132, 2607:f8b0:400c:c03::84
Connecting to doc-0s-18-docs.googleusercontent.com (doc-0s-18-docs.googleusercontent.com)|172.217.1

In [3]:
menu_df = pd.read_csv("menu.csv")
menu_df

,Pizza Toppings,Side,Drink
0,Pepperoni,Chicken Wedges,Cola Regular Large
1,Chicken strips,Garlic Bread,Orange Cola
2,Sweetcorn,Potato Wedges,Sugar-Free Cola
3,Ham,Salad Bowl,Lemonade
4,Mushrooms,Cheesy Garlic Tear and Share,Bottled Water
5,Onions,Mac n Cheese,NaN
6,Pineapple,Cheesy Garlic Bread,NaN
7,Smoked Bacon Rashers,Cheese Triangles,NaN
8,Green and Red Peppers,Cheesy Nachos,NaN
9,Jalapeno Peppers,Breaded Chicken Strips,NaN


In [ ]:
ent_classes = ["name", "postcode", "time", "topping", "side", "drink", "cardinal"]

In [ ]:
toppings = menu_df["Pizza Toppings"].tolist()
sides = menu_df["Side"].tolist()
drinks = menu_df["Drink"].tolist()

In [ ]:
toppings = [topping.lower() for topping in toppings if str(topping) != 'nan']
sides = [side.lower() for side in sides if str(side) != 'nan']
drinks = [drink.lower() for drink in drinks if str(drink) != 'nan']

In [ ]:
menu_entities = toppings + sides + drinks

In [ ]:
time_12 = r"((1[0-2]|0?[1-9]):([0-5][0-9]) ?([AaPp][Mm]))"
time_24 = r"([0-9]|0[0-9]|1[0-9]|2[0-3]):[0-5][0-9]"
postcode = r"^[a-zA-Z]{1,2}[0-9][0-9A-Za-z]{0,1} {0,1}[0-9][A-Za-z]{2}$"

# Model

In [4]:
!pip install -U spacy

     |████████████████████████████████| 6.2 MB 4.7 MB/s 
     |████████████████████████████████| 10.1 MB 44.7 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 457 kB 65.4 MB/s 
     |████████████████████████████████| 660 kB 73.2 MB/s 
     |████████████████████████████████| 181 kB 56.3 MB/s 
     |████████████████████████████████| 58 kB 5.9 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    U

In [5]:
!python -m spacy download en_core_web_trf

     |████████████████████████████████| 460.3 MB 22 kB/s 
     |████████████████████████████████| 51 kB 111 kB/s 
     |████████████████████████████████| 1.1 MB 9.9 MB/s 
     |████████████████████████████████| 3.8 MB 40.9 MB/s 
     |████████████████████████████████| 880 kB 54.0 MB/s 
     |████████████████████████████████| 84 kB 3.1 MB/s 
     |████████████████████████████████| 6.6 MB 40.7 MB/s 
     |████████████████████████████████| 596 kB 58.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=008339287fc25037ed75ebfe95a40f515adeec15d6facaef19a818d7ee9e6f91
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web

In [6]:
import spacy

In [7]:
nlp = spacy.load("en_core_web_trf", disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])

In [11]:
import pickle
pickle.dump(nlp, open('drive/MyDrive/COM3029 coursework 2/Main/Components/ner.pkl','wb'))

In [ ]:
def ner_ml(query):
  pred = []
  ents = nlp(query).ents

  for ent in ents:
    label = -1
    if ent.label_ == "PERSON":
      label = ent_classes.index('name')
    elif ent.label_ == "TIME":
      label = ent_classes.index('time')
    elif ent.label_ == "CARDINAL":
      label = ent_classes.index('cardinal')
    
    if label != -1:
      pred.append((ent.text, label))

  return pred

In [ ]:
def ner_h(query, n, cutoff):
  if n <= 0:
    n = 1

  tokens = query.split(' ')
  tokens = [token.lower() for token in tokens]
  tokens = [(''.join([char for char in token if str.isalnum(char) or char in ['-', '&', ' ']])) for token in tokens]

  n_grams = []

  for i in range(len(tokens)):
    for j in range(2, n+1):
      n_grams.append(' '.join(tokens[i:i+j]))

  tokens += n_grams

  entities = []

  for token in tokens:
    ent_class = -1
    ent_token = ""

    toppings_match = get_close_matches(token, toppings, n=1, cutoff=cutoff)
    sides_match = get_close_matches(token, sides, n=1, cutoff=cutoff)
    drinks_match = get_close_matches(token, drinks, n=1, cutoff=cutoff)

    toppings_ratio = 0
    sides_ratio = 0
    drinks_ratio = 0

    if toppings_match:
      toppings_ratio = SequenceMatcher(None, token, toppings_match[0]).ratio()
    if sides_match:
      sides_ratio = SequenceMatcher(None, token, sides_match[0]).ratio()
    if drinks_match:
      drinks_ratio = SequenceMatcher(None, token, drinks_match[0]).ratio()
    
    choices = ['topping', 'side', 'drink']
    ratios = [toppings_ratio, sides_ratio, drinks_ratio]
    matches = [toppings_match, sides_match, drinks_match]

    choice = ratios.index(max(ratios))

    if ratios[choice] > 0:
      ent_class = ent_classes.index(choices[choice])
      ent_token = matches[choice][0]

    if ent_class == -1:
      postcode_match = re.search(postcode, token)
      if postcode_match:
        ent_class = ent_classes.index('postcode')
        ent_token = postcode_match.group()

    if ent_class >= 0: entities.append((ent_token, ent_class))
  
  i = 0
  j = i + 1
  while j < len(entities):
    if entities[i][0] in entities[j][0]:
      entities.pop(i)
    elif entities[j][0] in entities[i][0]:
      entities.pop(j)
    i += 1
    j += 1

  entities = list(dict.fromkeys(entities))

  i = 0
  j = i + 1
  while j < len(entities):
    if entities[i][0] in entities[j][0]:
      entities.pop(i)
    elif entities[j][0] in entities[i][0]:
      entities.pop(j)
    i += 1
    j += 1

  return entities

In [ ]:
def get_entities(query):
  return ner_h(query, 5, 0.87) + ner_ml(query)

# References

1. https://www.delftstack.com/howto/python/remove-special-characters-from-string-python/

2. https://digitalfortress.tech/tips/top-15-commonly-used-regex/

3. https://spacy.io/models

4. https://regexlib.com/REDetails.aspx?regexp_id=38

5. https://www.kaggle.com/datasets/fivethirtyeight/fivethirtyeight-most-common-name-dataset?select=new-top-firstNames.csv

6. https://www.kaggle.com/datasets/manshanawaz/uk-postcode-and-geolocation-dataset?select=1a+Dataset+UK+Postcodes+and+Geolocation+CSV